In [4]:
!pip install selenium pandas requests
!pip install pillow

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import time
import requests
import pandas as pd
from selenium.webdriver.common.by import By
from PIL import Image
from io import BytesIO
from datetime import datetime
import csv 



In [11]:
driver_path = "C:/chromedriver2/chromedriver.exe"
service = Service(driver_path)
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=service, options=options)

driver.get("https://www.pinterest.com/login/")

try:
    email_input = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
        (By.NAME, "id")  
    ))
    email_input.send_keys("finalprojectphase1.2@gmail.com")  
    ## finalprojectphase1.1@gmail.com
    ## finalprojectphase1.2@gmail.com
    ## finalprojectphase1.3@gmail.com
    ## finalprojectphase1.4@gmail.com

except:
    print("⚠️ Failed to find email input field. Check the XPath or class.")


try:
    password_input = driver.find_element(By.NAME, "password")
    password_input.send_keys("final12345")
except:
    print("⚠️ Failed to find password input field.")


try:
    login_button = driver.find_element(By.CLASS_NAME, "RCK")
    login_button.click()
    print("✅ Logging in...")
except:
    print("⚠️ Failed to find login button. Trying another method...")
    login_button = driver.find_element(By.XPATH, "//button[contains(text(), 'Log in')]")
    login_button.click()


time.sleep(1)


if "login" not in driver.current_url:
    print("❌ Login failed. Check your email/password.")
else:
    print("✅ Logged in successfully!")


✅ Logging in...
✅ Logged in successfully!


In [12]:

base_folder = r"C:\Users\Lenovo\Desktop\DS\final_project\P0"
pinterest_folder = os.path.join(base_folder, "pinterest_dogs1")
os.makedirs(pinterest_folder, exist_ok=True)

breeds = ["Rottweiler", "Dalmatian", "Golden Retriever", "Pug"]
image_limit_per_breed = 500

csv_file = os.path.join(base_folder, "dog_images_data_table.csv")
header = ["Breed", "Image File Name", "Image File Path", "Image Size (KB)", "Download Time", "Resolution", "Image Format", "Source URL", "Status", "Dominant Color"]

def get_dominant_color(image_data):
    img = Image.open(BytesIO(image_data))
    img = img.resize((100, 100)) 
    img = img.convert('RGB')
    colors = img.getcolors(img.size[0] * img.size[1])
    dominant_color = max(colors, key=lambda item: item[0])[1]
    return dominant_color

def scrape_images_for_breed(breed):
    with open(csv_file, mode='a', newline='') as file:
        writer = csv.writer(file)
        if file.tell() == 0:
            writer.writerow(header)

        print(f"\n🔍 Searching for high-quality images of: {breed}")

        search_box = WebDriverWait(driver, 15).until(EC.presence_of_element_located(
            (By.XPATH, "//input[@aria-activedescendant]")  
        ))

        search_box.send_keys(Keys.CONTROL + "a")
        search_box.send_keys(Keys.BACKSPACE)
        search_box.send_keys(breed + " dog")
        search_box.send_keys(Keys.ENTER)

        time.sleep(10)

        folder_path = os.path.join(pinterest_folder, breed.replace(" ", "_"))
        os.makedirs(folder_path, exist_ok=True)

        image_urls = set()
        image_count = 0

        while image_count < image_limit_per_breed:
            driver.execute_script("window.scrollBy(0, 1000);")
            time.sleep(2)

            try:
                pins = driver.find_elements(By.XPATH, "//a[contains(@href, '/pin/')]")
                pin_links = [pin.get_attribute("href") for pin in pins if pin.get_attribute("href")]

                for pin_link in pin_links:
                    if image_count >= image_limit_per_breed:
                        break
                    
                    if pin_link in image_urls:
                        continue

                    image_urls.add(pin_link)

                    driver.execute_script(f"window.open('{pin_link}', '_blank');")
                    driver.switch_to.window(driver.window_handles[1])
                    time.sleep(5)

                    try:
                        full_image = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
                            (By.XPATH, "//img[contains(@src, 'pinimg.com')]")
                        ))

                        image_url = full_image.get_attribute("src")
                        image_source_url = pin_link

                        response = requests.get(image_url)

                        if response.status_code == 200:
                            img = Image.open(BytesIO(response.content))
                            width, height = img.size
                            resolution = f"{width}x{height}"

                            if width <= 75 and height <= 75:
                                print(f"🚫 Skipping profile picture pin: {pin_link} ({resolution})")
                                driver.close() 
                                driver.switch_to.window(driver.window_handles[0])
                                continue

                            image_size = len(response.content)
                            image_size_kb = round(image_size / 1024, 2)

                            download_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

                            image_format = img.format

                            dominant_color = get_dominant_color(response.content)
                            dominant_color_hex = "#%02x%02x%02x" % dominant_color

                            image_file_name = f"image_{image_count + 1}.jpg"
                            image_path = os.path.join(folder_path, image_file_name)

                            with open(image_path, 'wb') as file:
                                file.write(response.content)

                            writer.writerow([breed, image_file_name, image_path, image_size_kb, download_time, resolution, image_format, image_source_url, "Success", dominant_color_hex])

                            print(f"✅ Downloaded Image {image_count + 1} for {breed} ({image_size_kb} KB, {resolution}, {image_format}, Dominant Color: {dominant_color_hex})")

                            image_count += 1
                        else:
                            print(f"⚠️ Failed to download {image_url}")

                    except Exception as e:
                        print(f"Error: {e}")

                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    time.sleep(1)

            except Exception as e:
                print(f"Error while processing pins: {e}")
                continue

        print(f"✅ Completed scraping for: {breed}")



In [13]:
scrape_images_for_breed(breed=breeds[0])


🔍 Searching for high-quality images of: Rottweiler


InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=135.0.7049.115)
Stacktrace:
	GetHandleVerifier [0x00007FF7C5AA5305+78597]
	GetHandleVerifier [0x00007FF7C5AA5360+78688]
	(No symbol) [0x00007FF7C58591AA]
	(No symbol) [0x00007FF7C5844EF5]
	(No symbol) [0x00007FF7C5869F04]
	(No symbol) [0x00007FF7C58DED0F]
	(No symbol) [0x00007FF7C58FEBB2]
	(No symbol) [0x00007FF7C58D6EC3]
	(No symbol) [0x00007FF7C58A03F8]
	(No symbol) [0x00007FF7C58A1163]
	GetHandleVerifier [0x00007FF7C5D4EEBD+2870973]
	GetHandleVerifier [0x00007FF7C5D49668+2848360]
	GetHandleVerifier [0x00007FF7C5D66943+2967875]
	GetHandleVerifier [0x00007FF7C5AC014A+188746]
	GetHandleVerifier [0x00007FF7C5AC842F+222255]
	GetHandleVerifier [0x00007FF7C5AAD284+111236]
	GetHandleVerifier [0x00007FF7C5AAD432+111666]
	GetHandleVerifier [0x00007FF7C5A93559+5465]
	BaseThreadInitThunk [0x00007FF8BAAA259D+29]
	Rtl

In [15]:
scrape_images_for_breed(breed=breeds[1])


🔍 Searching for high-quality images of: Dalmatian
✅ Downloaded Image 1 for Dalmatian (43.23 KB, 560x840, JPEG, Dominant Color: #e1c8b2)
✅ Downloaded Image 2 for Dalmatian (109.48 KB, 736x1308, JPEG, Dominant Color: #fefefe)
✅ Downloaded Image 3 for Dalmatian (18.82 KB, 236x354, JPEG, Dominant Color: #d6d6cc)
✅ Downloaded Image 4 for Dalmatian (87.97 KB, 736x981, JPEG, Dominant Color: #d9bc9e)
✅ Downloaded Image 5 for Dalmatian (161.84 KB, 735x919, JPEG, Dominant Color: #a7a6a1)
✅ Downloaded Image 6 for Dalmatian (14.04 KB, 236x419, JPEG, Dominant Color: #948d73)
✅ Downloaded Image 7 for Dalmatian (9.61 KB, 236x236, JPEG, Dominant Color: #aba595)
✅ Downloaded Image 8 for Dalmatian (87.49 KB, 735x802, JPEG, Dominant Color: #e1c1aa)
✅ Downloaded Image 9 for Dalmatian (38.65 KB, 736x736, JPEG, Dominant Color: #e0d2c9)
✅ Downloaded Image 10 for Dalmatian (11.94 KB, 236x236, JPEG, Dominant Color: #ffffff)
✅ Downloaded Image 11 for Dalmatian (71.58 KB, 736x955, JPEG, Dominant Color: #8697a7)

In [6]:
scrape_images_for_breed(breed=breeds[2])


🔍 Searching for high-quality images of: Golden Retriever
✅ Downloaded Image 1 for Golden Retriever (66.0 KB, 736x1308, JPEG, Dominant Color: #ded9d3)
✅ Downloaded Image 2 for Golden Retriever (112.2 KB, 736x1123, JPEG, Dominant Color: #11110f)
✅ Downloaded Image 3 for Golden Retriever (61.43 KB, 612x877, JPEG, Dominant Color: #b79371)
✅ Downloaded Image 4 for Golden Retriever (27.19 KB, 236x419, JPEG, Dominant Color: #0c0d12)
🚫 Skipping profile picture pin: https://www.pinterest.com/pin/1092545190865139729/ (75x75)
✅ Downloaded Image 5 for Golden Retriever (88.07 KB, 720x702, JPEG, Dominant Color: #b9bdc1)
✅ Downloaded Image 6 for Golden Retriever (21.52 KB, 236x416, JPEG, Dominant Color: #2b2622)
✅ Downloaded Image 7 for Golden Retriever (35.16 KB, 362x544, JPEG, Dominant Color: #cbcad0)
✅ Downloaded Image 8 for Golden Retriever (26.0 KB, 236x295, JPEG, Dominant Color: #bca509)
✅ Downloaded Image 9 for Golden Retriever (116.99 KB, 736x981, JPEG, Dominant Color: #ccdbf0)
✅ Downloaded 

In [4]:
scrape_images_for_breed(breed=breeds[3])


🔍 Searching for high-quality images of: Pug
✅ Downloaded Image 1 for Pug (38.67 KB, 626x939, JPEG, Dominant Color: #f1ebef)
✅ Downloaded Image 2 for Pug (47.06 KB, 612x816, JPEG, Dominant Color: #ffffff)
✅ Downloaded Image 3 for Pug (50.98 KB, 720x900, JPEG, Dominant Color: #847e72)
✅ Downloaded Image 4 for Pug (16.58 KB, 236x419, JPEG, Dominant Color: #dedede)
✅ Downloaded Image 5 for Pug (53.39 KB, 720x900, JPEG, Dominant Color: #c7c0b0)
✅ Downloaded Image 6 for Pug (116.02 KB, 736x981, JPEG, Dominant Color: #5890cd)
✅ Downloaded Image 7 for Pug (62.92 KB, 675x1200, JPEG, Dominant Color: #b8b9be)
✅ Downloaded Image 8 for Pug (12.48 KB, 236x295, JPEG, Dominant Color: #2e2926)
✅ Downloaded Image 9 for Pug (53.71 KB, 720x720, JPEG, Dominant Color: #eeeee4)
✅ Downloaded Image 10 for Pug (99.53 KB, 720x1077, JPEG, Dominant Color: #b2af9c)
✅ Downloaded Image 11 for Pug (18.33 KB, 236x295, JPEG, Dominant Color: #c4bfc5)
✅ Downloaded Image 12 for Pug (53.0 KB, 720x942, JPEG, Dominant Color: